Import stuff I need

In [1]:
import pyarrow.parquet as pq
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os


Upload into memory the data files. They are in parquet format.

In [2]:
table_sales = pq.read_table('../data/raw/modeldata.parquet')

In [3]:
table = pq.read_table('../data/raw/assmntdata.parquet')

table_sales includes data for homes in Cook County that have been sold in the past seven years. table includes data for all homes regardless of a sale. 

Convert to pandas dataframes.

In [4]:
df_sales = table_sales.to_pandas()

In [5]:
df = table.to_pandas()


In [6]:
df.head()

,meta_pin,meta_year,meta_class,meta_town_code,meta_nbhd,meta_2yr_pri_board_est_bldg,meta_2yr_pri_board_est_land,meta_1yr_pri_board_est_bldg,meta_1yr_pri_board_est_land,meta_mailed_est_bldg,...,time_sale_month_of_year,time_sale_week_of_year,time_sale_day_of_year,time_sale_during_school_year,time_sale_during_holidays,ind_multi_code,ind_large_home,ind_class_error,ind_garage,ind_complete_predictors
0,01011000040000,2020.0,212,10,10012,374930.0,52510.0,320210.0,58030.0,320210.0,...,Jan,1.0,1.0,True,True,False,False,False,True,True
1,01011000050000,2020.0,205,10,10012,216890.0,31030.0,275500.0,34300.0,275500.0,...,Jan,1.0,1.0,True,True,False,False,False,True,True
2,01011000060000,2020.0,212,10,10012,353260.0,31030.0,296390.0,34300.0,296390.0,...,Jan,1.0,1.0,True,True,False,False,False,True,True
3,01011000090000,2020.0,212,10,10012,668790.0,64460.0,604870.0,71240.0,604870.0,...,Jan,1.0,1.0,True,True,False,False,False,True,True
4,01011000170000,2020.0,205,10,10012,291310.0,42870.0,355640.0,47390.0,355640.0,...,Jan,1.0,1.0,True,True,False,False,False,False,True


examine columns

In [ ]:
df_sales.info()

In [ ]:
df.info()

On-line documentation describes some but not all columns. 


df_sales columns should be the same as df except that df_sales also includes sales columns. Let's check.

In [ ]:
df_sales_set = set(df_sales.columns)

In [ ]:
df_set = set(df.columns)

In [ ]:
print(df_sales_set.difference(df_set))

In [ ]:
print(df_set.difference(df_sales_set))

meta_sales_price is the important column. For my simplified model I'll scrap the others.

Let's focus on homes for the same elementary school in Cook County. Alternatives, use median income as proxy for school ranking. The single district we will focus on is the Bell School district becuase it has a lot of bungaloes and mansions mixed together so it is a district where regression to the mean could be present in the County's model.

In [ ]:
df['geo_school_elem_district'].unique()


In [ ]:
df_bell = df[df['geo_school_elem_district']=='BELL']

In [ ]:
df_bell.info()

In [ ]:
df_bell_sales = df_sales[df_sales['geo_school_elem_district']=='BELL']

In [ ]:
df_bell_sales.info()

The features I keep, in additon to the unque pin identifier for each property, as well as year and class of the property are the numeric ones rather than categories: land square footage, age, # rooms, #  bedrooms, # fireplaces, # full bathrooms, # half bathrooms, "ot_impr", "building square footage, logitude, and latitude. All of these are self explanatory except "ot_impr" which I might drop if I can't figure out from the documentation what it means. All of these features relate to the property except for the logitude and latitude, which relate to location. Prof Nick Tideman, an economics professor at Virgina Tech, whose research is on land values tells me that the proper way to model this system is to model land values seperately from building values and then add them together. To do this I would have to seperatly identify those properties that are flipped, and assume they are sold at their land value alone. But for my first attempt at this, I will model the land plus property as a single price which is what Cook County does. 

In [ ]:
df_bell_gf = df_bell.iloc[:,[0,1,2,21,22,26,27,33,35,36,45,46,74,75]]


In [ ]:
df_bell_gf.head()

In [ ]:
df_bell_sales_gf = df_bell_sales.iloc[:,[0,1,2,19,23,24,28,29,35,37,38,47,48,76,77]]

In [ ]:
df_bell_sales_gf.head()

In [ ]:
df_bell_gf.info()


In [ ]:
df_bell_sales_gf.info()

All values are non-null expect for 1 entry in # fireplaces. But it is only missing in the sales file. I should be able fill this in from the other file. In fact, the feature values in the sales file should be identical for the same property id's in the main file. Privided the property hasn't flipped. Let check this for 1 pin.

In [ ]:
df_bell_gf[df_bell_gf.meta_pin=='13242020170000'].T

In [ ]:
df_bell_sales_gf[df_bell_sales_gf.meta_pin=='13242020170000'].T

The year (presumably of the data) and the age is different but consistant with each other. The same pin can have differnet years which should effect the age of the property only. Except that if a house is sold and flipped then then other property feautures should change too.

Just to verify that all values are present:



In [ ]:
missing = pd.concat([df_bell_gf.isnull().sum(), 100 * df_bell_gf.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count',ascending=False)


check that each pin number in the two files is unique


In [ ]:
df_bell_gf['meta_pin'].value_counts().head()

We have four duplicates. Lets look at them.


In [ ]:
df_bell_gf[df_bell_gf['meta_pin']=='14191240230000']

In [ ]:
df_bell_gf[df_bell_gf['meta_pin']=='14192020110000']

In [ ]:
df_bell_gf[df_bell_gf['meta_pin']=='14191050190000']

In [ ]:
df_bell_gf[df_bell_gf['meta_pin']=='14192170310000']

It is not clear what is going on here. Let's check the sales file.

In [ ]:
df_bell_sales_gf['meta_pin'].value_counts().head(100)

It makes sense that there should be lots of duplicates in the sales file since properties are sold many times. Let's take a look at a few.


In [ ]:
df_bell_sales_gf[df_bell_sales_gf['meta_pin']=='14191050270000']


This tells a story that I partially understand. The property was sold in 2014. The old house was torn down and a new house built in its place or the old house was significantly renovated. The new property was then sold in 2015. This explains the drop in age from 106 to 1. The property was sold again in 2018. the features are the same in 2015 and 2018. Between 2014 and 2015 the feautures changed, consistant with the flipping scenerio. However the feauture changes are weird. the totakl number of rooms and the total number of bed rooms goes down and but the number of full and half bathrooms go up.  Also, the building square footage goes down. Despit an enormous change in price.Clearly the data is not accurate. Let's try another.


In [ ]:
df_bell_sales_gf[df_bell_sales_gf['meta_pin']=='13242080030000']

This is consistant with a flip. The age goes down to 1 the price goes way up and the features change. This time all the features change in the right direction. Now I confirm that pins are unique if I suppliment them by year and class.


In [ ]:
(df_bell_sales_gf['meta_pin'] + ', ' + df_bell_sales_gf['meta_year'].astype('str')+ ', ' + df_bell_sales_gf['meta_class']).value_counts().head()


What is ot_impr?

In [ ]:
df_bell_sales_gf['char_ot_impr'].value_counts()

It means nothing. I can drop it.

In [ ]:
df_bell_sales_gf.nunique()

In [ ]:
df_bell_gf.nunique()

All other catergories, besides ot_impr convey some information, except that the year in the main file is set to 2020 for every property. The main file also includes more property classes. Just for fun I'll look at value counts for the property classes in a bar graph.


In [ ]:

df_bell_gf.meta_class.value_counts().plot(kind='barh', title="Counts of Classes", xlabel="Class")



In [ ]:
df_bell_sales_gf.meta_class.value_counts().plot(kind='barh', title="Counts of Classes", xlabel="Class")

There are some classes that are different in each file. Is the class a feature worth keeping or is all the class information included in the building square footage, age, and land square footage? Let's check.

In [ ]:
df_bell_sales_gf.groupby('meta_class')[['char_bldg_sf','char_hd_sf','char_age']].mean()

In [ ]:
df_bell_gf.groupby('meta_class')[['char_bldg_sf','char_hd_sf','char_age']].mean()


The data is constant with the definitions of these classes. 202-204 are single story with increasing minimum building square footage. 205-206 are two or more stories and older than 62 years. 207-278-208 are two or more story and less than or equal to 62 years with increasing building square footage. They are out of numerical order. 211 are apartmment buildings with 2-6 appartments. I think this must include condos and 2 and 3 flats. They are a dominant older property in the district. 212 is mixed use residential and comerical with comercial on the ground floor. 2-34 is split level with lower level below ground. 295 is townhome or row house. Will need to study distribution in each class to see if classes can be dropped as the data is represented by the features. As no feauture represents number of stories, I might need to retain this if I decide that this is important. 


In [ ]:
df_bell_gf.describe().T

In [ ]:
df_bell_sales_gf.describe().T

Will need to check some of these. The maximums for the main file seem to be errors: 46 rooms, 24 bed rooms, 6 fireplaces, 10 bathroom, 6 half baths. These are supposed to be individual residendences. Some of the maximums for the sales file are equally outlandish: 42 rooms, 24 bedrooms, 6 bathrooms, 6 half baths. 

In [ ]:
df_bell_sales_gf.hist(figsize=(15,10))
plt.subplots_adjust(hspace=0.5);

There appear to be gap years in the sales data. There is only slight evidence for a bimodal sales distribution from slight dip less than 1.5 million. But the age is clearly bimodal. There appear to be outliers in hd_sf, rooms, beds, half baths, and bld_sf, 

In [ ]:
df_bell_gf.hist(figsize=(15,10))
plt.subplots_adjust(hspace=0.5);

outliers in rooms, fbaths, hbaths, ot_impr, and bldg_sf

In [ ]:
df_bell_sales_gf[df_bell_sales_gf.char_rooms >= 20].T

In [ ]:
df_bell_gf[df_bell_gf.char_rooms > 21].T

All of the ones with huge numbers of rooms are in class 211 which is "Apartment building with 2 to 6 units, any age" It looks like the building is being treated as a whole. In any case, it is sufficiently different than a single residece that I should not inlcude class 211. Here's how it looks with them.

In [ ]:
df_bell_sales_gf[df_bell_sales_gf.meta_class != '211'].hist(figsize=(15,10))
plt.subplots_adjust(hspace=0.5);

In [ ]:
df_bell_gf[df_bell_gf.meta_class != '211'].hist(figsize=(15,10))
plt.subplots_adjust(hspace=0.5);

Let's check out the remaining outlier 

In [ ]:
df_bell_sales_gf[(df_bell_sales_gf.meta_class != '211') & (df_bell_sales_gf.char_hd_sf > 6000)].T

In [ ]:
df_bell_gf[(df_bell_gf.meta_class != '211') & (df_bell_gf.char_hd_sf > 6000)].T

Aside from one entry, these are old buildings. In some cases wealthy people have bought to adjacent properties and tore down one to use the land as a yard for the other. I wonder if this explains the huge land sf and smaller bldg sqwuare footage for some entries. Or they could be errors in entry.

In [ ]:
df_bell_sales_gf[(df_bell_sales_gf.meta_class != '211') & (df_bell_sales_gf.char_bldg_sf > 6000)].T

In [ ]:
df_bell_gf[(df_bell_gf.meta_class != '211') & (df_bell_gf.char_bldg_sf > 6000)].T

All of these are class 212 "Mixed-use commercial/residential building with apartment and commercial area totaling 6 units or less with a square foot area less than 20,000 square feet, any age" and should be removed.

In [ ]:
bell = df_bell_gf[(df_bell_gf.meta_class != '211') & (df_bell_gf.meta_class != '212')]

In [ ]:
bell_sales = df_bell_sales_gf[(df_bell_sales_gf.meta_class != '211') & (df_bell_sales_gf.meta_class != '212')]

In [ ]:
bell.info()

In [ ]:
bell_sales.info()

637 single family residences and 293 sales prices to work with.

In [ ]:
bell.hist(figsize=(15,10))
plt.subplots_adjust(hspace=0.5);

In [ ]:
bell_sales.hist(figsize=(15,10))
plt.subplots_adjust(hspace=0.5);

A possible bimodel distribution in price is a little more visible now. Let's check out ot_impr.


In [ ]:
bell[bell.char_ot_impr > 0].T

Can't tell what it is but because there is no info in sales data it is useless to keep.

In [ ]:
bell = bell.drop(columns = 'char_ot_impr')

In [ ]:
bell_sales= bell_sales.drop(columns = 'char_ot_impr')

In [ ]:
bell.head()


In [ ]:
bell_sales.head()

In [ ]:
bell.to_csv('../data/bell_cleaned.csv')

In [ ]:
bell_sales.to_csv('../data/bell_sales_cleaned.csv')

I have saved single family home data in the Bell district. I have retained numerical features plus class.